# Device Driver Architecture

{numref}`fig:iobus:driver` illustrates the I/O process for a typical
device from user-space application request through the driver, hardware
I/O operation, interrupt, and finally back to user space.

```{figure} ../images/pb-figures/devs/iobus-driver.png
---
width: 70% 
name: fig:iobus:driver
---
Driver Architecture
```

In more detail:

- The user process executes a `read` system call, which in turn invokes
the driver `read` operation, found via the `read` method of the file
operations structure.

- The driver fills in a DMA descriptor (in motherboard RAM), writes the
physical address of the descriptor to a device register (generating a
Memory Write operation across the PCIe bus), and then goes to sleep.

- The device issues a PCIe Memory Read Multiple command to read the DMA
descriptor from RAM.

- The device does some sort of I/O. (e.g. read from a disk, or receive a
network packet)

- A Memory Write and Invalidate operation is used to write the received
data back across the PCIe bus to the motherboard RAM, and to tell the
CPU to invalidate any cached copies of those addresses.

- A hardware interrupt from the device causes the device driver interrupt
handler to run.

- The interrupt handler wakes up the original process, which is currently
in kernel space in the device driver read method, in a call to something
like `interruptible_sleep_on`. After waking up, the read method copies
the data to the user buffer and returns.